# iflearner client Configure and start

## 1. Virtual environment
You can create a virtual running environment of iflearner through virtual tools such as conda and pyenv, and then activate it. 

### 1.1. Conda
1. You need to install conda environment. [install](https://docs.conda.io/projects/conda/en/stable/user-guide/install/index.html)

2. We will create a virtual environment through conda and activate it   
```shell
$ conda create -n iflearner python==3.9 ipykernel
$ conda activate iflearner
```

3. Write the virtual environment into the kernel of jupyter notebook        
command: python -m ipykernel install --user --name virtual-environment-name --display-name virtual-environment-name 
- The first virtual environment name indicates the created virtual environment name
- The second virtual environment name indicates that you want it to appear in the kernel options of the jupyter noteboook

  example: $ python -m ipykernel install --user --name iflearner --display-name "iflearner"

4. switch kernel
You can select the change kernel button in the kernel menu bar of jupyterlab and select the virtual environment we created

## 2. Install the iflearner library and related dependencies

In [1]:
!pip install iflearner torch==1.7.1 torchvision==0.8.2  --index-url http://pypi.douban.com/simple --trusted-host pypi.douban.com 

Looking in indexes: http://pypi.douban.com/simple


## 3. Define Client and Start

### 3.1. Define Pytorch model

In [1]:
import torch
import torch.nn.functional as F
import torch.optim as optim
from torch import nn
from torchvision import datasets, transforms

from iflearner.business.homo.argument import parser
from iflearner.business.homo.pytorch_trainer import PyTorchTrainer
from iflearner.business.homo.train_client import Controller


class Model(nn.Module):
    def __init__(self, num_channels, num_classes):
        super().__init__()
        self.conv1 = nn.Conv2d(num_channels, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, num_classes)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, x.shape[1] * x.shape[2] * x.shape[3])
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

### 3.2. Integrate and implement PytorchTrainer

In [2]:
class Mnist(PyTorchTrainer):
    def __init__(self, lr=0.15, momentum=0.5) -> None:
        self._lr = lr
        self._device = (
            torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
        )
        print(f"device: {self._device}")
        self._model = Model(num_channels=1, num_classes=10).to(self._device)

        super().__init__(self._model)

        self._optimizer = optim.SGD(self._model.parameters(), lr=lr, momentum=momentum)
        self._loss = F.nll_loss

        apply_transform = transforms.Compose(
            [transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))]
        )
        train_dataset = datasets.MNIST(
            "./data", train=True, download=True, transform=apply_transform
        )
        test_dataset = datasets.MNIST(
            "./data", train=False, download=True, transform=apply_transform
        )
        self._train_data = torch.utils.data.DataLoader(
            train_dataset, batch_size=64, shuffle=True
        )
        self._test_data = torch.utils.data.DataLoader(
            test_dataset, batch_size=64, shuffle=False
        )

    def fit(self, epoch):
        self._model.to(self._device)
        self._model.train()
        print(
            f"Epoch: {epoch}, the size of training dataset: {len(self._train_data.dataset)}, batch size: {len(self._train_data)}"
        )
        for batch_idx, (data, target) in enumerate(self._train_data):
            data, target = data.to(self._device), target.to(self._device)
            self._optimizer.zero_grad()
            output = self._model(data)
            loss = self._loss(output, target)
            loss.backward()
            self._optimizer.step()

    def evaluate(self, epoch):
        self._model.to(self._device)
        self._model.eval()
        test_loss = 0
        correct = 0
        print(f"The size of testing dataset: {len(self._test_data.dataset)}")
        with torch.no_grad():
            for data, target in self._test_data:
                data, target = data.to(self._device), target.to(self._device)
                output = self._model(data)
                test_loss += self._loss(
                    output, target, reduction="sum"
                ).item()  # sum up batch loss
                pred = output.argmax(
                    dim=1, keepdim=True
                )  # get the index of the max log-probability
                correct += pred.eq(target.view_as(pred)).sum().item()
        test_loss /= len(self._test_data.dataset)

        print(
            "Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)".format(
                test_loss,
                correct,
                len(self._test_data.dataset),
                100.0 * correct / len(self._test_data.dataset),
            )
        )

        return {"loss": test_loss, "acc": correct / len(self._test_data.dataset)}

### 3.3. Start client

In [ ]:
if __name__ == "__main__":
    args = parser.parse_args(args=[])
    print(args)
    args.name = "client02"
    args.epochs = 2
    mnist = Mnist()
    controller = Controller(args, mnist)
    controller.run()

2022-08-08 15:10:20.752 | INFO     | iflearner.business.homo.train_client:run:89 - register to server


Namespace(cert=None, enable_ll=0, epochs=10, name='client', peers=None, server='localhost:50001')
device: cpu


2022-08-08 15:10:20.925 | INFO     | iflearner.communication.homo.homo_client:transport:59 - OUT: message type: msg_register, time: 171.61533600000035ms
2022-08-08 15:10:20.927 | INFO     | iflearner.business.homo.train_client:run:106 - use strategy: FedAvg
2022-08-08 15:10:20.932 | INFO     | iflearner.business.homo.train_client:run:139 - report client ready
2022-08-08 15:10:20.968 | INFO     | iflearner.communication.homo.homo_client:transport:59 - OUT: message type: msg_client_ready, time: 33.54244400000095ms
2022-08-08 15:10:21.943 | INFO     | iflearner.communication.homo.homo_client:notice:94 - IN: party: message type: msg_notify_training, time: 3.377977999999615ms
2022-08-08 15:10:21.975 | INFO     | iflearner.business.homo.train_client:run:149 - ----- fit <FT> -----


Epoch: 1, the size of training dataset: 60000, batch size: 938


2022-08-08 15:10:48.314 | INFO     | iflearner.business.homo.train_client:run:167 - ----- evaluate <FT> -----


...

Epoch: 10, the size of training dataset: 60000, batch size: 938


2022-08-08 15:15:24.933 | INFO     | iflearner.business.homo.train_client:run:167 - ----- evaluate <FT> -----


The size of testing dataset: 10000


2022-08-08 15:15:27.289 | INFO     | iflearner.business.homo.train_client:run:178 - ----- get <FT> -----


Test set: Average loss: 0.0570, Accuracy: 9830/10000 (98.30%)
label: FT, points: ([1, 2, 3, 4, 5, 6, 7, 8, 9, 10], [0.12315296365953982, 0.09213094894941896, 0.0810126814592164, 0.06658224524520337, 0.06041985516438726, 0.06403173434020719, 0.07068393242153106, 0.06073516227027867, 0.08081894085274544, 0.056957239710199795])
label: LT, points: ([1], [0.12315296365953982])
label: FT, points: ([1, 2, 3, 4, 5, 6, 7, 8, 9, 10], [0.9645, 0.9728, 0.9765, 0.9792, 0.982, 0.9796, 0.9796, 0.9809, 0.9735, 0.983])
label: LT, points: ([1], [0.9645])
